# Python Tuflow Tools - Xarray Accessor

**The XARRAY addon is in development!**

The functions provided in the other modules, e.g., `FvExtractor (from tfv.extractor import FvExtractor)` are STABLE.

For the STABLE tools that look very similar to the MATLAB tools - see "tfv_tools_low_level.ipynb"

Expect bugs :) Please feel free to open issues, with good descriptions, to here: https://gitlab.com/tuflow/tfv (see Issues tab on left).

In [ ]:
import tfv.xarray
import xarray as xr

# Opening files
We treat our TUFLOW-FV result like any normal xarray object. 

**warning:** Old versions of TUFLOW-FV have non-compatible "TIME" variable unit description for xarray. To fix this, simply add `decode_times=False` if you're workign with old results. New versions of TUFLOW-FV will work directly with xarray :) 

In [ ]:
# NOTE - Python only likes "/". Long story short, if you want to copy paste in WINDOWS paths, just put an "r" infront of the string, e.g., see below. 
ds = xr.open_dataset(r"C:\Users\alexander.waterhouse\Python\tfv\working\HYD_002_extended.nc", decode_times=False)

In [ ]:
ds

However we can access a bunch of TUFLOW-FV Specific functions by then running `.tfv`

In [ ]:
ds.tfv

Conceptually - this "accessor" just adds on a bunch of functions, and helps you look at the results. 
All the hard work is being done by the existing tools available in `FvExtractor`, or the plotting functions in `Visual`. 

# Common Examples

## Spatial Plots

By default, the tool will plot the first variable it finds, with the first timestep it finds.

In [ ]:
ds.tfv.plot()

You can add lots of arguments to style plots to your liking. 
There is also some magic behind the scenes - if you have `V_x` and `V_y` in your dataset, you can request `V` or `VDir` directly. 

In [ ]:
ds.tfv.plot('V', '2011-02-05 02:00', cmap='turbo', clim=(0, 0.5), datum='depth')

# Other things to try:
# Depth-averaging, e.g., datum=["sigma", "height", "depth"], combined with limits
# Shading = ["interp", "patch", "contour"]

# Time can be provided as a str, a Timestamp object, or a simple integer.

You can compose plots as per normal matplotlib methods. This will hopefully make preparing reports for figures easier :) 

In [ ]:
import matplotlib.pyplot as plt

Here is a very simple example and are just some common building blocks you might use. 

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(12, 6), constrained_layout=True)

ts = 50

ax = axes[0]
p1 = ds.tfv.plot('H', ts, ax=ax, shading='interp', colorbar=False)
ax.set_title('Surf-ele')
plt.colorbar(p1.patch, ax=ax, orientation='horizontal')

ax = axes[1]
p2 = ds.tfv.plot('V', ts, ax=ax,  shading='interp', colorbar=False)
v2 = ds.tfv.plot_vector('V', ts, ax=ax, color='w')
ax.set_title('Velocity')
plt.colorbar(p2.patch, ax=ax, orientation='horizontal')

ax = axes[2]
p3 = ds.tfv.plot('TEMP', ts, ax=ax, shading='interp', colorbar=False)
ax.set_title('Temperature')
plt.colorbar(p3.patch, ax=ax, orientation='horizontal')

## Timeseries

In [ ]:
locs = {
    'P1': (159.1, -31.39), 
    'P2': (159.115, -31.395), 
}

ts = ds.tfv.get_timeseries(['H', 'V', 'TEMP'], locs)
ts

In [ ]:
ts['V'][:, 1].plot()

## Sheet Timeseries

What if you want to make your own wild function that works on the sheet (2D) results? 

In [ ]:
# This grabs ALL timesteps between the 2nd to 4th of Feb 2011, and takes the average of the top 2m! 
sht = ds.tfv.get_sheet('SAL', slice('2011-02-02', '2011-02-04'), datum='depth', limits=(0,2))
sht

## Built-in statistics

We are always doing the same set of statistics - so they've been built in. Here's an example

In [ ]:
# List of percentiles to grab, as well as some common stats
stats = ['p5', 'p50', 'p97.5', 'p99', 'min', 'mean', 'max']

# Do stats on velocity, for the bottom 1m (default of limits is (0, 1), thus 1m). 
stat = ds.tfv.get_statistics(stats, 'V', datum='height')
stat

Now we can plot them

In [ ]:
stat.plot('V_p99', cmap='turbo', clim=(0,0.5))

## Export to QGIS

You can chop down a tfv result and send to QGIS at any time. 

Simply run `ds.tfv.to_netcdf('my_tfv_result.nc')` on any of the results and it should be all good - including from the statistics results, etc. 

# Interactive Plotting

Lastly, if you want to investigate your results interactively, you can use matplotlib's widget mode. 

First turn it on with `%matplotlib widget`. THIS WILL MAKE THE CHANGE FOR THE ENTIRE NOTEBOOK. 

To turn it off, run `%matplotlib inline`

In [ ]:
%matplotlib widget

In [ ]:
ds.tfv.plot_interactive('V', cmap='turbo', clim=(0,0.5), datum='depth', vectors=True)

# Other features

I'm way behind with documentation, but in the meantime, if you think there should be a way to do something, there probably is! 

Try typing `ds.tfv.` into a new cell, and then pressing "tab" to see a list of functions that are available to try (e.g., `plot_curtain`)

If you have new ideas, let me (A.Waterhouse) know.

Note, these tools are PUBLIC - and hence we won't be adding anything specific workflows. Only simple methdos to do basic things like assistance with plotting and getting results.

If you have a BMT Specific workflow you want to save / share - that's what this git repo is for. It should be an internal resource for sharing examples. 